In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter

import functools

import math

import matplotlib.pyplot as plt

In [46]:
inputFile = "ID_data_mass_18122012_prepared.csv"

dataFrame = pd.read_csv(inputFile, header = 0, sep = ';')
print(dataFrame.shape)
data = dataFrame.values
# print(data)

numSamples = dataFrame.shape[0]
print("Num samples == " + str(numSamples))
numFeatures = dataFrame.shape[1]  #Last two features used as class designation
print("Num features == " + str(numFeatures))

kgfColumnId = dataFrame.shape[1] - 1
gtotalColumnId = dataFrame.shape[1] - 2

def getClassMarker(row):
    return str(row[-2]) + '/' + str(row[-1])

(185, 33)
Num samples == 185
Num features == 33


In [48]:
def giniCoefficient(groups, classes):
    numOfSamples = float(sum([len(group) for group in groups]))
    
    # Total Gini index
    result = 0.0
    
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
            
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [getClassMarker(row) for row in group].count(class_val) / size
            score += p * p
            
        groupWeight = (size / numOfSamples)
        result += (1.0 - score) * groupWeight
    return result

print(giniCoefficient([[[1, 1], [1, 0]], 
                       [[1, 1], [1, 0]]], 
                      [0, 1]))
print(giniCoefficient([[[1, 0], [1, 0]], 
                       [[1, 1], [1, 1]]], 
                      [0, 1]))

1.0
1.0


In [27]:
def splitData(featureIndex, splitValue, data):
    left = list()
    right = list()
    
    for row in data:
        if row[featureIndex] < splitValue:
            left.append(row)
        else:
            right.append(row)
            
    return left, right

print(splitData(0,
                1,
                [[0, 1], [2, 0]]))

([[0, 1]], [[2, 0]])


In [50]:
# print (list(set(classMarker(row) for row in data)))

def findBestSplit(data, numNonClassMarkerFeatures):
    classMarkers = list(set(getClassMarker(row) for row in data))
    
    bestFeatureId = 0
    bestSplitValue = 0
    bestGiniCoeff = 999999
    bestSplit = None
    
    print(classMarkers)
    
    for featureId in range(numNonClassMarkerFeatures):
        for row in data:
            splittedData = splitData(featureId, row[featureId], data)
            coeff = giniCoefficient(splittedData, classMarkers)
            
            print('X%d < %.3f Gini=%.3f' % ((featureId + 1), row[featureId], coeff))
            
            if (coeff < bestGiniCoeff):
                bestFeatureId = featureId
                bestSplitValue = row[featureId]
                bestGiniCoeff = coeff
                bestSplit = splittedData
                
    return {'featureId' : bestFeatureId,
            'splitValue' : bestSplitValue,
            'dataGroups' : bestSplit}

testDataset = [[0.5, 1, 0],
               [1, 1, 0],
               [1.5, 2, 0],
               [2, 2, 0]]
split = findBestSplit(testDataset, 1)
print('Split: [X%d < %.3f]' % ((split['featureId']+1), split['splitValue']))

['2/0', '1/0']
X1 < 0.500 Gini=0.500
X1 < 1.000 Gini=0.333
X1 < 1.500 Gini=0.000
X1 < 2.000 Gini=0.333
Split: [X1 < 1.500]
